AES BUTTERFLY EFFECT ENC DEC


In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import numpy as np

def butterfly_effect(key):
    ks = sum(ord(char) for char in key)
    x = (ks % 8) + 1
    y = x + 1
    z = x + 1
    sigma = 10
    rho = 28
    beta = 8.0/3.0
    dt = 0.01

    butterfly_effect_values = []
    for i in range(len(key)):
        dx = (sigma * (y-x)) * dt
        dy = (x * (rho-z)) * dt
        dz = (x*y-beta*z)*dt
        x += dx
        y += dy
        z += dz
        butterfly_effect_values.append(int(x) ^ int(y) ^ int(z))

    return butterfly_effect_values

def add_butterfly_effect(state, round_key, round):
    for i in range(4):
        for j in range(len(state[i])):
            temp = (butterfly_effect[round*len(state[i]) + j] >> (3-i)) * 8
            temp %= 256
            temp = temp if temp >= 0 else 256+temp
            state[i][j] ^= temp
    return state

def aes_encrypt(plaintext, key):
    cipher = AES.new(key.encode(), AES.MODE_ECB)
    ciphertext = cipher.encrypt(pad(plaintext.encode(), AES.block_size))
    return ciphertext

def aes_decrypt(ciphertext, key):
    cipher = AES.new(key.encode(), AES.MODE_ECB)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return plaintext.decode()
# Example usage
key = "mysecretkey"  # 11 bytes
key = key.ljust(16, '0')  # Pad the key with zeros to make it 16 bytes long

plaintext = "Hello, World!"

# Encrypt
ciphertext = aes_encrypt(plaintext, key)
print("Ciphertext:", ciphertext)

# Decrypt
decrypted_text = aes_decrypt(ciphertext, key)
print("Decrypted text:", decrypted_text)

Ciphertext: b'8\xf0\xa6\xe9e\xeb\xa2}\x120\xcb\xec\x1d\xb5\x9e_'
Decrypted text: Hello, World!


In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import numpy as np
import random

def aes_encrypt(plaintext, key):
    cipher = AES.new(key.encode(), AES.MODE_ECB)
    ciphertext = cipher.encrypt(pad(plaintext.encode(), AES.block_size))
    return ciphertext

def aes_decrypt(ciphertext, key):
    cipher = AES.new(key.encode(), AES.MODE_ECB)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return plaintext.decode()

def bit_diff(a, b):
    return bin(a ^ b).count('1')

def calculate_confusion_diffusion(plaintext, key):
    # Encrypt the original plaintext
    ciphertext_original = aes_encrypt(plaintext, key)

    # Generate a new plaintext with one bit flipped
    plaintext_bytes = bytearray(plaintext.encode())
    random_index = random.randint(0, len(plaintext_bytes) - 1)
    plaintext_bytes[random_index] ^= 0x01  # Flip one bit
    plaintext_flipped = plaintext_bytes.decode()

    # Encrypt the modified plaintext
    ciphertext_flipped = aes_encrypt(plaintext_flipped, key)

    # Calculate the number of different bits between the two ciphertexts
    diff_bits = sum(bit_diff(a, b) for a, b in zip(ciphertext_original, ciphertext_flipped))

    # Total number of bits
    total_bits = len(ciphertext_original) * 8

    # Calculate diffusion percentage
    diffusion_percentage = (diff_bits / total_bits) * 100

    # For confusion, we measure how different the ciphertexts are
    # when the plaintexts are different
    ciphertext_different_plaintext = aes_encrypt("DifferentPlaintext", key)
    diff_bits_confusion = sum(bit_diff(a, b) for a, b in zip(ciphertext_original, ciphertext_different_plaintext))

    # Calculate confusion percentage
    confusion_percentage = (diff_bits_confusion / total_bits) * 100

    return confusion_percentage, diffusion_percentage

# Example usage
key = "Thats my Kung Fu"  # 11 bytes
key = key.ljust(16, '0')  # Pad the key with zeros to make it 16 bytes long

plaintext = "This is a sample text for AES encryption."

confusion, diffusion = calculate_confusion_diffusion(plaintext, key)
print("Confusion percentage:", confusion)
print("Diffusion percentage:", diffusion)


Confusion percentage: 32.29166666666667
Diffusion percentage: 15.885416666666666


In [ ]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import random

# AES S-box and inverse S-box assumed to be defined
# sbox and sbox_inv are assumed to be defined as 2D numpy arrays
sbox = np.array([
    [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76],
    [0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0],
    [0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15],
    [0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75],
    [0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84],
    [0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf],
    [0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8],
    [0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2],
    [0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73],
    [0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb],
    [0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79],
    [0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08],
    [0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a],
    [0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e],
    [0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf],
    [0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]
])
sbox_inv = np.array([
    [0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb],
    [0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb],
    [0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e],
    [0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25],
    [0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92],
    [0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84],
    [0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06],
    [0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b],
    [0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73],
    [0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e],
    [0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b],
    [0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4],
    [0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f],
    [0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef],
    [0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61],
    [0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d]
])
# Butterfly Effect parameters
sigma = 10
rho = 28
beta = 8.0 / 3.0
dt = 0.01

def lorenz(x, y, z):
    max_iterations = 10000
    iteration = 0

    while iteration < max_iterations:
        dx = sigma * (y - x) * dt
        dy = (x * (rho - z) - y) * dt
        dz = (x * y - beta * z) * dt

        if abs(x) > 1e6 or abs(y) > 1e6 or abs(z) > 1e6:
            return x, y, z

        x += dx
        y += dy
        z += dz

        iteration += 1

    return x, y, z

def add_butterfly_effect(key):
    x, y, z = sum(bytearray(key)), sum(bytearray(key)), sum(bytearray(key))
    for i in range(len(key)):
        x, y, z = lorenz(x, y, z)
        x_index = int(x) % 16
        y_index = int(y) % 16
        key[i] ^= sbox[x_index][y_index]

def key_expansion(key):
    key_symbols = [ord(symbol) for symbol in key]
    add_butterfly_effect(key_symbols)
    key_schedule = np.zeros((4, 44), dtype=int)
    for r in range(4):
        for c in range(4):
            key_schedule[r][c] = key_symbols[r + 4 * c]
    for col in range(4, 44):
        if col % 4 == 0:
            temp = key_schedule[:, col - 1]
            temp = np.roll(temp, -1)
            temp = np.vectorize(lambda x: sbox[x // 16 % 16, x % 16])(temp)
            temp[0] ^= (1 << (col // 4 - 1))
            key_schedule[:, col] = (key_schedule[:, col - 4] ^ temp) % 256
        else:
            key_schedule[:, col] = (key_schedule[:, col - 4] ^ key_schedule[:, col - 1]) % 256
    return key_schedule

def galois_mult(a, b):
    p = 0
    for i in range(8):
        if b & 1:
            p ^= a
        high_bit_set = a & 0x80
        a <<= 1
        if high_bit_set:
            a ^= 0x1b
        b >>= 1
    return p & 0xff

def sub_bytes(state):
    return np.vectorize(lambda x: sbox[(x // 16) % 16, x % 16])(state)

def shift_rows(state):
    for i in range(1, 4):
        state[i] = np.roll(state[i], -i)
    return state

def mix_columns(state):
    for i in range(4):
        column = state[:, i]
        state[:, i] = [
            galois_mult(0x02, column[0]) ^ galois_mult(0x03, column[1]) ^ column[2] ^ column[3],
            column[0] ^ galois_mult(0x02, column[1]) ^ galois_mult(0x03, column[2]) ^ column[3],
            column[0] ^ column[1] ^ galois_mult(0x02, column[2]) ^ galois_mult(0x03, column[3]),
            galois_mult(0x03, column[0]) ^ column[1] ^ column[2] ^ galois_mult(0x02, column[3])
        ]
    return state

def add_round_key(state, key_schedule, round):
    for i in range(4):
        state[:, i] ^= key_schedule[:, round * 4 + i]
    return state

def encrypt_block(block, key_schedule):
    state = np.array(block).reshape(4, 4).T
    state = add_round_key(state, key_schedule, 0)
    for round in range(1, 10):
        state = sub_bytes(state)
        state = shift_rows(state)
        state = mix_columns(state)
        state = add_round_key(state, key_schedule, round)
    state = sub_bytes(state)
    state = shift_rows(state)
    state = add_round_key(state, key_schedule, 10)
    return state.T.reshape(16) % 256

def decrypt_block(block, key_schedule):
    state = np.array(block).reshape(4, 4).T
    state = add_round_key(state, key_schedule, 10)
    state = inv_shift_rows(state)
    state = inv_sub_bytes(state)
    for round in range(9, 0, -1):
        state = add_round_key(state, key_schedule, round)
        state = inv_mix_columns(state)
        state = inv_shift_rows(state)
        state = inv_sub_bytes(state)
    state = add_round_key(state, key_schedule, 0)
    return state.T.reshape(16) % 256

def inv_sub_bytes(state):
    return np.vectorize(lambda x: sbox_inv[(x // 16) % 16, x % 16])(state)

def inv_mix_columns(state):
    for i in range(4):
        column = state[:, i]
        state[:, i] = [
            galois_mult(0x0e, column[0]) ^ galois_mult(0x0b, column[1]) ^ galois_mult(0x0d, column[2]) ^ galois_mult(0x09, column[3]),
            galois_mult(0x09, column[0]) ^ galois_mult(0x0e, column[1]) ^ galois_mult(0x0b, column[2]) ^ galois_mult(0x0d, column[3]),
            galois_mult(0x0d, column[0]) ^ galois_mult(0x09, column[1]) ^ galois_mult(0x0e, column[2]) ^ galois_mult(0x0b, column[3]),
            galois_mult(0x0b, column[0]) ^ galois_mult(0x0d, column[1]) ^ galois_mult(0x09, column[2]) ^ galois_mult(0x0e, column[3])
        ]
    return state

def inv_shift_rows(state):
    for i in range(1, 4):
        state[i] = np.roll(state[i], i)
    return state

def bit_diff(a, b):
    return bin(a ^ b).count('1')

def calculate_confusion_diffusion(plaintext, key):
    key_schedule = key_expansion(key)

    blocks = [plaintext[i:i+16] for i in range(0, len(plaintext), 16)]
    ciphertext = []

    for block in blocks:
        block = [ord(char) for char in block]
        encrypted_block = encrypt_block(block, key_schedule)
        ciphertext.extend(encrypted_block)

    # Convert ciphertext to byte array
    ciphertext_bytes = bytearray([byte % 256 for byte in ciphertext])

    # Generate a new plaintext with one bit flipped
    plaintext_bytes = bytearray(plaintext.encode())
    random_index = random.randint(0, len(plaintext_bytes) - 1)
    plaintext_bytes[random_index] ^= 0x01  # Flip one bit
    plaintext_flipped = plaintext_bytes.decode()

    blocks_flipped = [plaintext_flipped[i:i+16] for i in range(0, len(plaintext_flipped), 16)]
    ciphertext_flipped = []

    for block in blocks_flipped:
        block = [ord(char) for char in block]
        encrypted_block = encrypt_block(block, key_schedule)
        ciphertext_flipped.extend(encrypted_block)

    # Convert flipped ciphertext to byte array
    ciphertext_flipped_bytes = bytearray([byte % 256 for byte in ciphertext_flipped])

    # Calculate the number of different bits between the two ciphertexts
    diff_bits = sum(bit_diff(a, b) for a, b in zip(ciphertext_bytes, ciphertext_flipped_bytes))

    # Total number of bits
    total_bits = len(ciphertext_bytes) * 8

    # Calculate diffusion percentage
    diffusion_percentage = (diff_bits / total_bits) * 100

    # For confusion, we measure how different the ciphertexts are
    # when the plaintexts are different
    different_plaintext = "CompletelyDiffText!"
    if len(different_plaintext) % 16 != 0:
        different_plaintext += ' ' * (16 - len(different_plaintext) % 16)

    blocks_different = [different_plaintext[i:i+16] for i in range(0, len(different_plaintext), 16)]
    ciphertext_different = []

    for block in blocks_different:
        block = [ord(char) for char in block]
        encrypted_block = encrypt_block(block, key_schedule)
        ciphertext_different.extend(encrypted_block)

    # Convert different ciphertext to byte array
    ciphertext_different_bytes = bytearray([byte % 256 for byte in ciphertext_different])

    diff_bits_confusion = sum(bit_diff(a, b) for a, b in zip(ciphertext_bytes, ciphertext_different_bytes))

    # Calculate confusion percentage
    confusion_percentage = (diff_bits_confusion / total_bits) * 100

    return confusion_percentage, diffusion_percentage

def main():
    plaintext = "This is a sample text for AES encryption."
    if len(plaintext) % 16 != 0:
        plaintext += ' ' * (16 - len(plaintext) % 16)

    key = "Thats my Kung Fu"
    confusion, diffusion = calculate_confusion_diffusion(plaintext, key)

    print("Confusion percentage:", confusion)
    print("Diffusion percentage:", diffusion)

if __name__ == "__main__":
    main()


Confusion percentage: 31.770833333333332
Diffusion percentage: 15.885416666666666


AES FULL CODE

In [ ]:

import numpy as np

# AES S-box
sbox = np.array([
    [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76],
    [0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0],
    [0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15],
    [0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75],
    [0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84],
    [0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf],
    [0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8],
    [0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2],
    [0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73],
    [0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb],
    [0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79],
    [0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08],
    [0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a],
    [0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e],
    [0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf],
    [0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]
])
sbox_inv = np.array([
    [0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb],
    [0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb],
    [0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e],
    [0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25],
    [0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92],
    [0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84],
    [0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06],
    [0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b],
    [0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73],
    [0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e],
    [0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b],
    [0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4],
    [0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f],
    [0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef],
    [0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61],
    [0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d]
])
# Rijndael's Galois field multiplication
def galois_mult(a, b):
    p = 0
    for i in range(8):
        if b & 1:
            p ^= a
        high_bit_set = a & 0x80
        a <<= 1
        if high_bit_set:
            a ^= 0x1b
        b >>= 1
    return p & 0xff

# Substitute bytes using the S-box
def sub_bytes(state):
    return np.vectorize(lambda x: sbox[(x // 16) % 16, x % 16])(state)

# Shift rows
def shift_rows(state):
    for i in range(1, 4):
        state[i] = np.roll(state[i], -i)
    return state

# Mix columns
# Mix columns
def mix_columns(state):
    for i in range(4):
        column = state[:, i]
        state[:, i] = [
            galois_mult(0x02, column[0]) ^ galois_mult(0x03, column[1]) ^ column[2] ^ column[3],
            column[0] ^ galois_mult(0x02, column[1]) ^ galois_mult(0x03, column[2]) ^ column[3],
            column[0] ^ column[1] ^ galois_mult(0x02, column[2]) ^ galois_mult(0x03, column[3]),
            galois_mult(0x03, column[0]) ^ column[1] ^ column[2] ^ galois_mult(0x02, column[3])
        ]
    return state

# Add round key
def add_round_key(state, key_schedule, round):
    for i in range(4):
        state[:, i] ^= key_schedule[:, round * 4 + i]
    return state


# Key expansion (simplified version)
def key_expansion(key):
    key_symbols = [ord(symbol) for symbol in key]
    key_schedule = np.zeros((4, 44), dtype=int)
    for r in range(4):
        for c in range(4):
            key_schedule[r][c] = key_symbols[r + 4 * c]
    for col in range(4, 44):
        if col % 4 == 0:
            temp = key_schedule[:, col - 1]
            temp = np.roll(temp, -1)
            temp = np.vectorize(lambda x: sbox[x // 16 % 16, x % 16])(temp)  # Fix index out of bounds
            temp[0] ^= (1 << (col // 4 - 1))
            key_schedule[:, col] = key_schedule[:, col - 4] ^ temp
        else:
            key_schedule[:, col] = key_schedule[:, col - 4] ^ key_schedule[:, col - 1]
    return key_schedule


# Encrypt a single block (16 bytes)
def encrypt_block_aes(block, key_schedule):
    state = np.array(block).reshape(4, 4).T
    state = add_round_key(state, key_schedule, 0)
    for round in range(1, 10):
        state = sub_bytes(state)
        state = shift_rows(state)
        state = mix_columns(state)
        state = add_round_key(state, key_schedule, round)
    state = sub_bytes(state)
    state = shift_rows(state)
    state = add_round_key(state, key_schedule, 10)
    return state.T.reshape(16)

# Decrypt a single block (16 bytes)
def decrypt_block(block, key_schedule):
    state = np.array(block).reshape(4, 4).T
    state = add_round_key(state, key_schedule, 10)
    state = inv_shift_rows(state)
    state = inv_sub_bytes(state)
    for round in range(9, 0, -1):
        state = add_round_key(state, key_schedule, round)
        state = inv_mix_columns(state)
        state = inv_shift_rows(state)
        state = inv_sub_bytes(state)
    state = add_round_key(state, key_schedule, 0)
    return state.T.reshape(16)

def inv_sub_bytes(state):
    return np.vectorize(lambda x: sbox_inv[(x // 16) % 16, x % 16])(state)

def inv_mix_columns(state):
    for i in range(4):
        column = state[:, i]
        state[:, i] = [
            galois_mult(0x0e, column[0]) ^ galois_mult(0x0b, column[1]) ^ galois_mult(0x0d, column[2]) ^ galois_mult(0x09, column[3]),
            galois_mult(0x09, column[0]) ^ galois_mult(0x0e, column[1]) ^ galois_mult(0x0b, column[2]) ^ galois_mult(0x0d, column[3]),
            galois_mult(0x0d, column[0]) ^ galois_mult(0x09, column[1]) ^ galois_mult(0x0e, column[2]) ^ galois_mult(0x0b, column[3]),
            galois_mult(0x0b, column[0]) ^ galois_mult(0x0d, column[1]) ^ galois_mult(0x09, column[2]) ^ galois_mult(0x0e, column[3])
        ]
    return state

def inv_shift_rows(state):
    for i in range(1, 4):
        state[i] = np.roll(state[i], i)
    return state

def main():
    plaintext = "This is a sample text for AES encryption."
    # Pad plaintext to be multiple of 16 bytes
    if len(plaintext) % 16 != 0:
        plaintext += ' ' * (16 - len(plaintext) % 16)

    key = "Thats my Kung Fu"
    key_schedule = key_expansion(key)

    blocks = [plaintext[i:i+16] for i in range(0, len(plaintext), 16)]
    ciphertext = []

    for block in blocks:
        block = [ord(char) for char in block]
        encrypted_block = encrypt_block_aes(block, key_schedule)
        ciphertext.extend(encrypted_block)

    print("Plaintext:", plaintext)
    print("Ciphertext (hex):", ''.join(format(x, '02x') for x in ciphertext))

    # Decryption
    decrypted_text = ""
    for i in range(0, len(ciphertext), 16):
        block = ciphertext[i:i+16]
        decrypted_block = decrypt_block(block, key_schedule)
        decrypted_text += ''.join(chr(byte) for byte in decrypted_block)

    print("Decrypted Text:", decrypted_text.strip())

if __name__ == "__main__":
    main()


Plaintext: This is a sample text for AES encryption.       
Ciphertext (hex): 313fa6a5c24af3c557363e7274e27872c4ff31cc560752762ee6f8355bbfb19210b3f47539cdd779f2d0b41a03340a8d94e2467b9e67
Decrypted Text: This is a sample text for AES encryption.


AES BUTTERFLY FULL CODE

In [ ]:
import numpy as np

# AES S-box
sbox = np.array([
    [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76],
    [0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0],
    [0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15],
    [0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75],
    [0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84],
    [0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf],
    [0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8],
    [0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2],
    [0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73],
    [0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb],
    [0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79],
    [0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08],
    [0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a],
    [0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e],
    [0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf],
    [0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]
])
sbox_inv = np.array([
    [0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb],
    [0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb],
    [0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e],
    [0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25],
    [0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92],
    [0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84],
    [0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06],
    [0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b],
    [0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73],
    [0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e],
    [0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b],
    [0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4],
    [0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f],
    [0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef],
    [0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61],
    [0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d]
])
import numpy as np

# Butterfly Effect parameters
sigma = 10
rho = 28
beta = 8.0 / 3.0
dt = 0.01

def lorenz(x, y, z):
    max_iterations = 10000  # Adjust this value as needed
    iteration = 0

    while iteration < max_iterations:
        dx = sigma * (y - x) * dt
        dy = (x * (rho - z) - y) * dt
        dz = (x * y - beta * z) * dt

        # Check if values exceed threshold
        if abs(x) > 1e6 or abs(y) > 1e6 or abs(z) > 1e6:
            return x, y, z

        x += dx
        y += dy
        z += dz

        iteration += 1

    return x, y, z




def add_butterfly_effect(key):
    x, y, z = sum(bytearray(key)), sum(bytearray(key)), sum(bytearray(key))
    for i in range(len(key)):
        x, y, z = lorenz(x, y, z)
        x_index = int(x) % 16
        y_index = int(y) % 16
        key[i] ^= sbox[x_index][y_index]  # Use adjusted indices to access S-box


# Key expansion with Butterfly Effect
def key_expansion(key):
    key_symbols = [ord(symbol) for symbol in key]
    add_butterfly_effect(key_symbols)  # Apply Butterfly Effect to initial key
    key_schedule = np.zeros((4, 44), dtype=int)
    for r in range(4):
        for c in range(4):
            key_schedule[r][c] = key_symbols[r + 4 * c]
    for col in range(4, 44):
        if col % 4 == 0:
            temp = key_schedule[:, col - 1]
            temp = np.roll(temp, -1)
            temp = np.vectorize(lambda x: sbox[x // 16 % 16, x % 16])(temp)  # Fix index out of bounds
            temp[0] ^= (1 << (col // 4 - 1))
            key_schedule[:, col] = key_schedule[:, col - 4] ^ temp
        else:
            key_schedule[:, col] = key_schedule[:, col - 4] ^ key_schedule[:, col - 1]
    return key_schedule
# Rijndael's Galois field multiplication
def galois_mult(a, b):
    p = 0
    for i in range(8):
        if b & 1:
            p ^= a
        high_bit_set = a & 0x80
        a <<= 1
        if high_bit_set:
            a ^= 0x1b
        b >>= 1
    return p & 0xff

# Substitute bytes using the S-box
def sub_bytes(state):
    return np.vectorize(lambda x: sbox[(x // 16) % 16, x % 16])(state)

# Shift rows
def shift_rows(state):
    for i in range(1, 4):
        state[i] = np.roll(state[i], -i)
    return state

# Mix columns
# Mix columns
def mix_columns(state):
    for i in range(4):
        column = state[:, i]
        state[:, i] = [
            galois_mult(0x02, column[0]) ^ galois_mult(0x03, column[1]) ^ column[2] ^ column[3],
            column[0] ^ galois_mult(0x02, column[1]) ^ galois_mult(0x03, column[2]) ^ column[3],
            column[0] ^ column[1] ^ galois_mult(0x02, column[2]) ^ galois_mult(0x03, column[3]),
            galois_mult(0x03, column[0]) ^ column[1] ^ column[2] ^ galois_mult(0x02, column[3])
        ]
    return state

# Add round key
def add_round_key(state, key_schedule, round):
    for i in range(4):
        state[:, i] ^= key_schedule[:, round * 4 + i]
    return state


# Key expansion (simplified version)



# Encrypt a single block (16 bytes)
def encrypt_block(block, key_schedule):
    state = np.array(block).reshape(4, 4).T
    state = add_round_key(state, key_schedule, 0)
    for round in range(1, 10):
        state = sub_bytes(state)
        state = shift_rows(state)
        state = mix_columns(state)
        state = add_round_key(state, key_schedule, round)
    state = sub_bytes(state)
    state = shift_rows(state)
    state = add_round_key(state, key_schedule, 10)
    return state.T.reshape(16)

# Decrypt a single block (16 bytes)
def decrypt_block(block, key_schedule):
    state = np.array(block).reshape(4, 4).T
    state = add_round_key(state, key_schedule, 10)
    state = inv_shift_rows(state)
    state = inv_sub_bytes(state)
    for round in range(9, 0, -1):
        state = add_round_key(state, key_schedule, round)
        state = inv_mix_columns(state)
        state = inv_shift_rows(state)
        state = inv_sub_bytes(state)
    state = add_round_key(state, key_schedule, 0)
    return state.T.reshape(16)

def inv_sub_bytes(state):
    return np.vectorize(lambda x: sbox_inv[(x // 16) % 16, x % 16])(state)

def inv_mix_columns(state):
    for i in range(4):
        column = state[:, i]
        state[:, i] = [
            galois_mult(0x0e, column[0]) ^ galois_mult(0x0b, column[1]) ^ galois_mult(0x0d, column[2]) ^ galois_mult(0x09, column[3]),
            galois_mult(0x09, column[0]) ^ galois_mult(0x0e, column[1]) ^ galois_mult(0x0b, column[2]) ^ galois_mult(0x0d, column[3]),
            galois_mult(0x0d, column[0]) ^ galois_mult(0x09, column[1]) ^ galois_mult(0x0e, column[2]) ^ galois_mult(0x0b, column[3]),
            galois_mult(0x0b, column[0]) ^ galois_mult(0x0d, column[1]) ^ galois_mult(0x09, column[2]) ^ galois_mult(0x0e, column[3])
        ]
    return state

def inv_shift_rows(state):
    for i in range(1, 4):
        state[i] = np.roll(state[i], i)
    return state

def main():
    plaintext = "This is a sample text for AES encryption."
    # Pad plaintext to be multiple of 16 bytes
    if len(plaintext) % 16 != 0:
        plaintext += ' ' * (16 - len(plaintext) % 16)

    key = "Thats my Kung Fu"
    key_schedule = key_expansion(key)

    blocks = [plaintext[i:i+16] for i in range(0, len(plaintext), 16)]
    ciphertext = []

    for block in blocks:
        block = [ord(char) for char in block]
        encrypted_block = encrypt_block(block, key_schedule)
        ciphertext.extend(encrypted_block)

    print("Plaintext:", plaintext)
    print("Ciphertext (hex):", ''.join(format(x, '02x') for x in ciphertext))

    # Decryption
    decrypted_text = ""
    for i in range(0, len(ciphertext), 16):
        block = ciphertext[i:i+16]
        decrypted_block = decrypt_block(block, key_schedule)
        decrypted_text += ''.join(chr(byte) for byte in decrypted_block)

    print("Decrypted Text:", decrypted_text.strip())

if __name__ == "__main__":
    main()


Plaintext: This is a sample text for AES encryption.       
Ciphertext (hex): 30e5d70bd2385a97e83142d7a232c3d98b62304e0b5f620a7e756135c88df012198df5253747f5f6e21515f1db39e2e74dc263bc6ba2
Decrypted Text: This is a sample text for AES encryption.


In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

def aes_encrypt(key, plaintext):
    # Generate a random IV (Initialization Vector)
    iv = os.urandom(16)

    # Create an AES cipher object
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())

    # Create an encryptor object
    encryptor = cipher.encryptor()

    # Encrypt the plaintext
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()

    return iv + ciphertext

def aes_decrypt(key, ciphertext):
    # Extract IV from the ciphertext
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]

    # Create an AES cipher object
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())

    # Create a decryptor object
    decryptor = cipher.decryptor()

    # Decrypt the ciphertext
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    return plaintext

def main():
    # Generate a random AES key (128, 192, or 256 bits)
    key = os.urandom(32)  # 32 bytes for AES-256

    plaintext = b"Hello, world!"
    print("Plain text:", plaintext)
    # Encrypt the plaintext
    ciphertext = aes_encrypt(key, plaintext)
    print("Ciphertext:", ciphertext.hex())

    # Decrypt the ciphertext
    decrypted_text = aes_decrypt(key, ciphertext)
    print("Decrypted text:", decrypted_text.decode())

if __name__ == "__main__":
    main()


Plain text: b'Hello, world!'
Ciphertext: 76127f55850f595ba5df8b33872294b626b25cf4b7c1fd67c6de17ffae
Decrypted text: Hello, world!


In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives import hashes
import os
import time
import psutil

def aes_encrypt(key, plaintext):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    padded_plaintext = pad(plaintext)
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    return iv + ciphertext

def aes_decrypt(key, ciphertext):
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()
    return unpad(decrypted_data)

# Padding functions
def pad(data):
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(data)
    padded_data += padder.finalize()
    return padded_data

def unpad(data):
    unpadder = padding.PKCS7(128).unpadder()
    unpadded_data = unpadder.update(data)
    unpadded_data += unpadder.finalize()
    return unpadded_data

def measure_encryption_speed(key, plaintext):
    start_time = time.time()
    ciphertext = aes_encrypt(key, plaintext)
    end_time = time.time()
    return end_time - start_time, len(ciphertext)

def measure_decryption_speed(key, ciphertext):
    start_time = time.time()
    plaintext = aes_decrypt(key, ciphertext)
    end_time = time.time()
    return end_time - start_time, len(plaintext)

def measure_memory_usage():
    process = psutil.Process()
    memory_usage = process.memory_info().rss
    return memory_usage

def main():
    key = os.urandom(32)  # AES-256 key
    plaintext = b"Hello, world!"

    # Measure encryption speed and memory usage
    encryption_time, ciphertext_size = measure_encryption_speed(key, plaintext)
    memory_usage_before_encryption = measure_memory_usage()

    # Measure decryption speed and memory usage
    decryption_time, plaintext_size = measure_decryption_speed(key, aes_encrypt(key, plaintext))
    memory_usage_after_decryption = measure_memory_usage()

    print("Encryption time:", encryption_time, "seconds")
    print("Decryption time:", decryption_time, "seconds")
    print("Memory usage before encryption:", memory_usage_before_encryption, "bytes")
    print("Memory usage after decryption:", memory_usage_after_decryption, "bytes")
    print("Ciphertext size:", ciphertext_size, "bytes")
    print("Plaintext size:", plaintext_size, "bytes")

if __name__ == "__main__":
    main()


Encryption time: 0.00031566619873046875 seconds
Decryption time: 0.00012063980102539062 seconds
Memory usage before encryption: 225763328 bytes
Memory usage after decryption: 225763328 bytes
Ciphertext size: 32 bytes
Plaintext size: 13 bytes


CONFUSION DIFFUSION PERCENTAGES OF AES BUTTERFLY

In [ ]:
import numpy as np
import time
from memory_profiler import profile

# AES S-box
# sbox and inv_sbox are defined here

# Butterfly Effect parameters
sigma = 10
rho = 28
beta = 8.0 / 3.0
dt = 0.01

# Lorenz function
def lorenz(x, y, z):
    max_iterations = 10000  # Adjust this value as needed
    iteration = 0

    while iteration < max_iterations:
        dx = sigma * (y - x) * dt
        dy = (x * (rho - z) - y) * dt
        dz = (x * y - beta * z) * dt

        # Check if values exceed threshold
        if abs(x) > 1e6 or abs(y) > 1e6 or abs(z) > 1e6:
            return x, y, z

        x += dx
        y += dy
        z += dz

        iteration += 1

    return x, y, z

# Function to add Butterfly Effect to the key
def add_butterfly_effect(key):
    x, y, z = sum(bytearray(key)), sum(bytearray(key)), sum(bytearray(key))
    for i in range(len(key)):
        x, y, z = lorenz(x, y, z)
        x_index = int(x) % 16
        y_index = int(y) % 16
        key[i] ^= sbox[x_index][y_index]

# Key expansion with Butterfly Effect
def key_expansion(key):
    key_symbols = [ord(symbol) for symbol in key]
    add_butterfly_effect(key_symbols)
    key_schedule = np.zeros((4, 44), dtype=int)
    for r in range(4):
        for c in range(4):
            key_schedule[r][c] = key_symbols[r + 4 * c]
    for col in range(4, 44):
        if col % 4 == 0:
            temp = key_schedule[:, col - 1]
            temp = np.roll(temp, -1)
            temp = np.vectorize(lambda x: sbox[x // 16 % 16, x % 16])(temp)
            temp[0] ^= (1 << (col // 4 - 1))
            key_schedule[:, col] = key_schedule[:, col - 4] ^ temp
        else:
            key_schedule[:, col] = key_schedule[:, col - 4] ^ key_schedule[:, col - 1]
    return key_schedule

# Rijndael's Galois field multiplication
def galois_mult(a, b):
    p = 0
    for i in range(8):
        if b & 1:
            p ^= a
        high_bit_set = a & 0x80
        a <<= 1
        if high_bit_set:
            a ^= 0x1b
        b >>= 1
    return p & 0xff

# Substitute bytes using the S-box
def sub_bytes(state):
    return np.vectorize(lambda x: sbox[(x // 16) % 16, x % 16])(state)

# Shift rows
def shift_rows(state):
    for i in range(1, 4):
        state[i] = np.roll(state[i], -i)
    return state

# Mix columns
def mix_columns(state):
    for i in range(4):
        column = state[:, i]
        state[:, i] = [
            galois_mult(0x02, column[0]) ^ galois_mult(0x03, column[1]) ^ column[2] ^ column[3],
            column[0] ^ galois_mult(0x02, column[1]) ^ galois_mult(0x03, column[2]) ^ column[3],
            column[0] ^ column[1] ^ galois_mult(0x02, column[2]) ^ galois_mult(0x03, column[3]),
            galois_mult(0x03, column[0]) ^ column[1] ^ column[2] ^ galois_mult(0x02, column[3])
        ]
    return state

# Add round key
def add_round_key(state, key_schedule, round):
    for i in range(4):
        state[:, i] ^= key_schedule[:, round * 4 + i]
    return state

# Encrypt a single block (16 bytes)
def encrypt_block(block, key_schedule):
    state = np.array(block).reshape(4, 4).T
    initial_state = state.copy()  # Store the initial state for confusion calculation
    state = add_round_key(state, key_schedule, 0)
    for round in range(1, 10):
        state = sub_bytes(state)
        state = shift_rows(state)
        state = mix_columns(state)
        state = add_round_key(state, key_schedule, round)
    state = sub_bytes(state)
    state = shift_rows(state)
    state = add_round_key(state, key_schedule, 10)

    # Calculate confusion percentage
    confusion_percentage = np.mean(np.abs(initial_state - state)) / 256 * 100

    return state.T.reshape(16), confusion_percentage
# Decrypt a single block (16 bytes)
def decrypt_block(block, key_schedule):
    state = np.array(block).reshape(4, 4).T
    initial_state = state.copy()  # Store the initial state for diffusion calculation
    state = add_round_key(state, key_schedule, 10)
    state = inv_shift_rows(state)
    state = inv_sub_bytes(state)
    for round in range(9, 0, -1):
        state = add_round_key(state, key_schedule, round)
        state = inv_mix_columns(state)
        state = inv_shift_rows(state)
        state = inv_sub_bytes(state)
    state = add_round_key(state, key_schedule, 0)

    # Calculate diffusion percentage
    diffusion_percentage = np.mean(np.abs(initial_state - state)) / 256 * 100

    return state.T.reshape(16), diffusion_percentage
# Inverse Substitute bytes using the Inverse S-box
def inv_sub_bytes(state):
    return np.vectorize(lambda x: sbox_inv[(x // 16) % 16, x % 16])(state)

# Inverse Mix columns
def inv_mix_columns(state):
    for i in range(4):
        column = state[:, i]
        state[:, i] = [
            galois_mult(0x0e, column[0]) ^ galois_mult(0x0b, column[1]) ^ galois_mult(0x0d, column[2]) ^ galois_mult(0x09, column[3]),
            galois_mult(0x09, column[0]) ^ galois_mult(0x0e, column[1]) ^ galois_mult(0x0b, column[2]) ^ galois_mult(0x0d, column[3]),
            galois_mult(0x0d, column[0]) ^ galois_mult(0x09, column[1]) ^ galois_mult(0x0e, column[2]) ^ galois_mult(0x0b, column[3]),
            galois_mult(0x0b, column[0]) ^ galois_mult(0x0d, column[1]) ^ galois_mult(0x09, column[2]) ^ galois_mult(0x0e, column[3])
        ]
    return state

# Inverse Shift rows
def inv_shift_rows(state):
    for i in range(1, 4):
        state[i] = np.roll(state[i], i)
    return state

@profile
def main():
    plaintext = "This is a sample text for AES encryption."
    # Pad plaintext to be multiple of 16 bytes
    if len(plaintext) % 16 != 0:
        plaintext += ' ' * (16 - len(plaintext) % 16)
    confusion_sum=0
    diffusion_sum=0
    key = "Thats my Kung Fu"
    key_schedule = key_expansion(key)

    blocks = [plaintext[i:i+16] for i in range(0, len(plaintext), 16)]
    ciphertext = []

    start_encryption = time.time()
    for block in blocks:
        block = [ord(char) for char in block]
        encrypted_block, confusion_percentage = encrypt_block(block, key_schedule)
        ciphertext.extend(encrypted_block)
        confusion_sum += confusion_percentage
    end_encryption = time.time()
    average_confusion = confusion_sum / len(blocks)
    print("Average Confusion Percentage:", average_confusion)
    encryption_time = end_encryption - start_encryption

    print("Plaintext:", plaintext)
    print("Ciphertext (hex):", ''.join(format(x, '02x') for x in ciphertext))

    # Decryption
    start_decryption = time.time()
    decrypted_text = ""
    for i in range(0, len(ciphertext), 16):
        block = ciphertext[i:i+16]
        decrypted_block, diffusion_percentage = decrypt_block(block, key_schedule)
        decrypted_text += ''.join(chr(byte) for byte in decrypted_block)
        diffusion_sum += diffusion_percentage
    end_decryption = time.time()

    decryption_time = end_decryption - start_decryption

    average_diffusion = diffusion_sum / len(blocks)

    print("Average Diffusion Percentage:", average_diffusion)
    print("Decrypted Text:", decrypted_text.strip())
    print("Encryption Time:", encryption_time, "seconds")
    print("Decryption Time:", decryption_time, "seconds")

if __name__ == "__main__":
    main()

ERROR: Could not find file <ipython-input-80-e5999d6d30ce>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.
Average Confusion Percentage: 82.99153645833333
Plaintext: This is a sample text for AES encryption.       
Ciphertext (hex): 30e5d70bd2385a97e83142d7a232c3d98b62304e0b5f620a7e756135c88df012198df5253747f5f6e21515f1db39e2e74dc263bc6ba2
Average Diffusion Percentage: 82.99153645833333
Decrypted Text: This is a sample text for AES encryption.
Encryption Time: 0.18490004539489746 seconds
Decryption Time: 0.38839101791381836 seconds


In [ ]:
!pip install memory_profiler
